In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import cv2
from imutils import perspective
import os
from tqdm.notebook import tqdm
import pandas as pd
from tqdm.notebook import tqdm
from src import config

In [ ]:
def gaussian2D(shape, sigma=1):
    m, n = [(ss - 1.) / 2. for ss in shape]
    y, x = np.ogrid[-m:m+1,-n:n+1]

    h = np.exp(-(x * x + y * y) / (2 * sigma * sigma))
    h[h < np.finfo(h.dtype).eps * h.max()] = 0
    return h

def draw_umich_gaussian(heatmap, center, radius, k=1):
    diameter = 2 * radius + 1
    gaussian = gaussian2D((diameter, diameter), sigma=diameter / 6)

    x, y = int(center[0]), int(center[1])

    height, width = heatmap.shape[0:2]

    left, right = min(x, radius), min(width - x, radius + 1)
    top, bottom = min(y, radius), min(height - y, radius + 1)

    masked_heatmap = heatmap[y - top:y + bottom, x - left:x + right]
    masked_gaussian = gaussian[radius - top:radius + bottom, radius - left:radius + right]
    if min(masked_gaussian.shape) > 0 and min(masked_heatmap.shape) > 0:  # TODO debug
        np.maximum(masked_heatmap, masked_gaussian * k, out=masked_heatmap)
    return masked_heatmap

In [ ]:
path = config.PATH_TO_TRAIN_IMAGES
with open(config.PATH_TO_ANNOTATION, 'rb') as f:
    annotations = pickle.load(f)

In [ ]:
paths_img = []
paths_hm = []

for file in os.listdir(path):
    bboxes = []
    centroids = []
    abs_path = os.path.join(path, file)
    slice_img = np.load(abs_path)
    hm = np.zeros((5, slice_img.shape[0], slice_img.shape[1]))
    
    for key in annotations[file.split('.')[0]].keys():
        (xc,yc),(w,h),angle = annotations[file.split('.')[0]][key]
        bbox = perspective.order_points(cv2.boxPoints(((xc,yc), (w, h), angle)))
        centroids.append((xc,yc))
        bboxes.append(bbox)
        
    for k in range(len(annotations[file.split('.')[0]].keys())):
        draw_umich_gaussian(hm[0,:,:], centroids[k], radius=8)
        draw_umich_gaussian(hm[1,:,:], bboxes[k][0], radius=8)
        draw_umich_gaussian(hm[2,:,:], bboxes[k][1], radius=8)
        draw_umich_gaussian(hm[3,:,:], bboxes[k][2], radius=8)
        draw_umich_gaussian(hm[4,:,:], bboxes[k][3], radius=8)
    
    paths_img.append(abs_path)
    paths_hm.append(os.path.join(config.PATH_TO_DUMP_HM, file))
    
    np.save(os.path.join(config.PATH_TO_DUMP_HM, file),np.array(hm))

In [ ]:
data = pd.DataFrame({'path_slice': paths_img, 'path_hm': paths_hm})

In [ ]:
data.to_csv(config.PATH_TO_DATA)

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(slice_img, cmap = 'gray')
plt.imshow(hm[1], cmap='hot', alpha=0.2)
plt.imshow(hm[2], cmap='hot', alpha=0.2)
plt.imshow(hm[3], cmap='hot', alpha=0.2)
plt.imshow(hm[4], cmap='hot', alpha=0.2)
plt.imshow(hm[0], cmap='hot', alpha=0.2)

In [ ]:
np.array(bboxes).shape

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(slice_img, cmap = 'gray')
plt.scatter(np.array(bboxes)[:,:,0],np.array(bboxes)[:,:,1])
plt.scatter(np.array(bboxes)[:,0,0],np.array(bboxes)[:,0,1], color='red')
plt.scatter(np.array(bboxes)[:,1,0],np.array(bboxes)[:,1,1], color='green')
plt.scatter(np.array(bboxes)[:,2,0],np.array(bboxes)[:,2,1], color='blue')
plt.scatter(np.array(bboxes)[:,3,0],np.array(bboxes)[:,3,1], color='yellow')
plt.scatter(np.array(centroids)[:,0], np.array(centroids)[:,1])